In [1]:
%config Completer.use_jedi = False
%matplotlib inline

In [119]:
import os
import pytest
import xarray as xr 

import datashader as ds

from datashader.transfer_functions import shade

from xrspatial.terrain import generate_terrain
from xrspatial import hillshade

import rasterio, rioxarray

from qgis.core import *
from qgis.PyQt.QtCore import QFileInfo, QVariant

canvas = ds.Canvas(plot_height=400, plot_width=800, x_range=(-10,10), y_range=(-10,10))
terrain = generate_terrain(canvas=canvas)


/Users/ls/anaconda3/envs/xrs/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
QgsApplication.setPrefixPath("/Users/ls/anaconda3/envs/xrs/bin/qgis", True)

qgs = QgsApplication([], True)

qgs.initQgis()

/Users/ls/anaconda3/envs/xrs/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [20]:
project = QgsProject.instance()

fileName = "raster_terrrain.tif"
fileInfo = QFileInfo(fileName)
baseName = fileInfo.baseName()
rlayer = QgsRasterLayer(fileName, baseName)
# listNames = [layer.name() for layer in project.mapLayers().values()]
# layersList = { }
# for l in project.mapLayers().values():
#     layersList[l.name()] = l

# root = project.layerTreeRoot()
from qgis.PyQt.Qt import QColor
fcn = QgsColorRampShader()
fcn.setColorRampType(QgsColorRampShader.Interpolated)
lst = [ QgsColorRampShader.ColorRampItem(0, QColor(0,0,0)), 
        QgsColorRampShader.ColorRampItem(255, QColor(255,255,255)) ]
fcn.setColorRampItemList(lst)
shader = QgsRasterShader()
shader.setRasterShaderFunction(fcn)

/Users/ls/anaconda3/envs/xrs/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# renderer = QgsSingleBandPseudoColorRenderer(rlayer.dataProvider(), 1, shader)
# rlayer.setRenderer(renderer)
# rlayer.save()

In [121]:
import numpy as np
from qgis.analysis import QgsRasterMatrix, QgsSlopeFilter, QgsAspectFilter
np.set_printoptions(threshold=10)
s = QgsHillshadeRenderer(rlayer.dataProvider(), 1, 25, 24)
QgsSlopeFilter()
# si = s.input()
# sib = si.block(1, s.extent(), rlayer.width(), rlayer.height())
# shape = (sib.height(), sib.width())
# out = np.zeros(shape, dtype=np.float)
# for y in range(shape[0]):
#     for x in range(shape[1]):
#         out[y, x] = sib.value(y, x)
# out
# sibd = sib.data
# sibd()

TypeError: QgsRasterMatrix(): arguments did not match any overloaded call:
  overload 1: too many arguments
  overload 2: argument 1 has unexpected type 'numpy.ndarray'

In [ ]:
# image_location = "render.png"

# vlayer = rlayer
# settings = QgsMapSettings()
# settings.setLayers([vlayer])
# settings.setBackgroundColor(QColor(255, 255, 255))
# settings.setOutputSize(QSize(800, 600))
# settings.setExtent(vlayer.extent())

# render = QgsMapRendererParallelJob(settings)

# def finished():
#     img = render.renderedImage()
#     # save the image; e.g. img.save("/Users/myuser/render.png","png")
#     img.save(image_location, "png")

# render.finished.connect(finished)

# # Start the rendering
# render.start()

# # The following loop is not normally required, we
# # are using it here because this is a standalone example.
# from qgis.PyQt.QtCore import QEventLoop
# loop = QEventLoop()
# render.finished.connect(loop.quit)
# loop.exec_()

aspect,
bump,
classify,
convolution,
curvature,
esri,
focal, 
hillshade,
multispectral,
pathfinding,
perlin,
proximity,
slope,
terrain,
tiles,
utils,
viewshed,
zonal